In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict

load_dotenv('/Users/wudawei/Documents/GitHub/LLM-test20251223/4.1 My LLM model/LLM_api.env')

True

In [2]:
class HelloAgentsLLM:
    """
    为本书 "Hello Agents" 定制的LLM客户端。
    它用于调用任何兼容OpenAI接口的服务，并默认使用流式响应。
    """
    def __init__(self, model: str = None, apiKey: str = None, baseUrl: str = None, timeout: int = None):
        """
        初始化客户端。优先使用传入参数，如果未提供，则从环境变量加载。
        """
        self.model = model or os.getenv("LLM_MODEL_ID")
        apiKey = apiKey or os.getenv("LLM_API_KEY")
        baseUrl = baseUrl or os.getenv("LLM_BASE_URL")
        timeout = timeout or int(os.getenv("LLM_TIMEOUT", 60))
        
        if not all([self.model, apiKey, baseUrl]):
            raise ValueError("模型ID、API密钥和服务地址必须被提供或在.env文件中定义。")

        self.client = OpenAI(api_key=apiKey, base_url=baseUrl, timeout=timeout)

    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        """
        调用大语言模型进行思考，并返回其响应。
        """
        print(f"🧠 正在调用 {self.model} 模型...")
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True,
            )
            
            # 处理流式响应
            print("✅ 大语言模型响应成功:")
            collected_content = []
            for chunk in response:
                content = chunk.choices[0].delta.content or ""
                print(content, end="", flush=True)
                collected_content.append(content)
            print()  # 在流式输出结束后换行
            return "".join(collected_content)

        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return None

# --- 客户端使用示例 ---
if __name__ == '__main__':
    try:

        llmClient = HelloAgentsLLM()
        
        exampleMessages = [
            {"role": "system", "content": "You are a helpful assistant that writes Python code."},
            {"role": "user", "content": "今天深圳天气怎么样？"}
        ]
        
        print("--- 调用LLM ---")
        responseText = llmClient.think(exampleMessages)
        if responseText:
            print("\n\n--- 完整模型响应 ---")
            print(responseText)

    except ValueError as e:
        print(e)


--- 调用LLM ---
🧠 正在调用 deepseek-ai/DeepSeek-R1-Distill-Qwen-32B 模型...
✅ 大语言模型响应成功:
要获取深圳今天的天气情况，可以使用天气API。以下是一个示例代码，使用中国气象局的API来获取深圳的天气数据：

```python
import requests

def get_weather(city_code):
    url = "http://api.weather预报.com/"  # 假设这是天气预报API的URL
    params = {
        "citycode": city_code,
        "key": "your_api_key"  # 替换为你的API密钥
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            # 提取今天天气的相关信息
            today = data["weather"][0]
            print(f"深圳今天天气：")
            print(f"温度：{today['temp']}℃")
            print(f"天气状况：{today['weather_desc']}")
            print(f"风力：{today['wind_desc']}")
        else:
            print(f"请求失败，状态码：{response.status_code}")
    except Exception as e:
        print(f"发生错误：{e}")

# 深圳的城市代码是101280601
get_weather("101280601")
```

注意：你需要替换`your_api_key`为实际的API密钥，并确保API的URL是正确的。如果你没有API密钥，可能需要注册一个天气API服务来获取。

运行这个代码后，它会打印出深圳今天的天气情况。


--- 完整

# 1. ReAct

In [30]:
import serpapi

def search(query: str) -> str:
    """
    一个基于SerpApi的实战网页搜索引擎工具。
    它会智能地解析搜索结果，优先返回直接答案或知识图谱信息。
    """
    print(f"🔍 正在执行 [SerpApi] 网页搜索: {query}")
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "错误:SERPAPI_API_KEY 未在 .env 文件中配置。"

        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
        }

        results = serpapi.search(params)
        print(results)
        # results = client.get_dict()
        
        # 智能解析:优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"


In [31]:
import serpapi
dir(serpapi)

['APIKeyNotProvided',
 'Client',
 'HTTPClient',
 'HTTPConnectionError',
 'HTTPError',
 'SearchIDNotProvided',
 'SerpApiError',
 'SerpResults',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'account',
 'core',
 'exceptions',
 'http',
 'locations',
 'models',
 'requests',
 'search',
 'search_archive',
 'textui']

In [32]:
from typing import Dict, Any

class ToolExecutor:
    """
    一个工具执行器，负责管理和执行工具。
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name: str, description: str, func: callable):
        """
        向工具箱中注册一个新工具。
        """
        if name in self.tools:
            print(f"警告:工具 '{name}' 已存在，将被覆盖。")
        self.tools[name] = {"description": description, "func": func}
        print(f"工具 '{name}' 已注册。")

    def getTool(self, name: str) -> callable:
        """
        根据名称获取一个工具的执行函数。
        """
        return self.tools.get(name, {}).get("func")

    def getAvailableTools(self) -> str:
        """
        获取所有可用工具的格式化描述字符串。
        """
        return "\n".join([
            f"- {name}: {info['description']}" 
            for name, info in self.tools.items()
        ])


In [33]:
# --- 工具初始化与使用示例 ---
if __name__ == '__main__':
    # 1. 初始化工具执行器
    toolExecutor = ToolExecutor()

    # 2. 注册我们的实战搜索工具
    search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
    toolExecutor.registerTool("Search", search_description, search)
    
    # 3. 打印可用的工具
    print("\n--- 可用的工具 ---")
    print(toolExecutor.getAvailableTools())

    # 4. 智能体的Action调用，这次我们问一个实时性的问题
    print("\n--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---")
    tool_name = "Search"
    tool_input = "英伟达最新的GPU型号是什么"

    tool_function = toolExecutor.getTool(tool_name)
    if tool_function:
        observation = tool_function(tool_input)
        print("--- 观察 (Observation) ---")
        print(observation)
    else:
        print(f"错误:未找到名为 '{tool_name}' 的工具。")

工具 'Search' 已注册。

--- 可用的工具 ---
- Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

--- 执行 Action: Search['英伟达最新的GPU型号是什么'] ---
🔍 正在执行 [SerpApi] 网页搜索: 英伟达最新的GPU型号是什么
{
    "search_metadata": {
        "id": "694dd2d3874d505464dcb3a7",
        "status": "Success",
        "json_endpoint": "https://serpapi.com/searches/ed711c37427976ed/694dd2d3874d505464dcb3a7.json",
        "pixel_position_endpoint": "https://serpapi.com/searches/ed711c37427976ed/694dd2d3874d505464dcb3a7.json_with_pixel_position",
        "created_at": "2025-12-26 00:12:03 UTC",
        "processed_at": "2025-12-26 00:12:03 UTC",
        "google_url": "https://www.google.com/search?q=%E8%8B%B1%E4%BC%9F%E8%BE%BE%E6%9C%80%E6%96%B0%E7%9A%84GPU%E5%9E%8B%E5%8F%B7%E6%98%AF%E4%BB%80%E4%B9%88&oq=%E8%8B%B1%E4%BC%9F%E8%BE%BE%E6%9C%80%E6%96%B0%E7%9A%84GPU%E5%9E%8B%E5%8F%B7%E6%98%AF%E4%BB%80%E4%B9%88&sourceid=chrome&ie=UTF-8",
        "raw_html_file": "https://serpapi.com/searches/ed711c37427976ed/694dd2d3874d505464dcb3a7.html